### Steam Recommendation System

Learning Problem:

With tens of thousands of games in their catalog, the steam store has been the at the center of digital game distribution around the world providing their users with remarkable services for decades. However, despite the size of their catalog, finding the right game can be difficult when they're are essentially unlimited options at your finger tip. Thus, we set out to see how we can approach this issue with machine learning lense. Specifically, experimenting with graph neural networks (GNN) for link prediction and how their applications can be applied to our problem to find relationship.

In [2]:
import pandas as pd
import numpy as np

Downloading datasets:

[Note] Tthe user review dataset as it is a 40+ GB file

In [ ]:
import kagglehub

# Download latest version
games_path = kagglehub.dataset_download("nikatomashvili/steam-games-dataset")

print("Path to games dataset files:", games_path)

reviews_path = kagglehub.dataset_download("kieranpoc/steam-reviews")

print("Path to reviews dataset files:", reviews_path)

/home/gordon/Code/Python/Machine_Learning/Final_Assignment/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to games dataset files: /home/gordon/.cache/kagglehub/datasets/nikatomashvili/steam-games-dataset/versions/1


Data Processing

In [ ]:
games_file = games_path + "/merged_data.csv"
steam_game_data = pd.read_csv(games_file)
steam_game_data = steam_game_data.dropna()
steam_game_data = steam_game_data[["Title", "Release Date", "Original Price", "Recent Reviews Summary", "All Reviews Summary", "Developer", "Popular Tags", "Game Features"]]
steam_game_data = steam_game_data.rename(columns={"Release Date": "Release Year"})
steam_game_data.head(5)

,Title,Release Year,Original Price,Recent Reviews Summary,All Reviews Summary,Developer,Popular Tags,Game Features
0,Baldur's Gate 3,"3 Aug, 2023",$29.99,Overwhelmingly Positive,Very Positive,Larian Studios,"['RPG', 'Choices Matter', 'Character Customiza...","['Single-player', 'Online Co-op', 'LAN Co-op',..."
1,Counter-Strike: Global Offensive,"21 Aug, 2012",$14.99,Very Positive,Very Positive,"Valve, Hidden Path Entertainment","['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['Steam Achievements', 'Full controller suppor..."
2,Apex Legends™,"4 Nov, 2020",Free,Mixed,Very Positive,Respawn Entertainment,"['Free to Play', 'Multiplayer', 'Battle Royale...","['Online PvP', 'Online Co-op', 'Steam Achievem..."
3,Forza Horizon 5,"8 Nov, 2021",$34.78,Very Positive,Very Positive,Playground Games,"['Racing', 'Open World', 'Driving', 'Multiplay...","['Single-player', 'Online PvP', 'Online Co-op'..."
4,Call of Duty®,"27 Oct, 2022",Free,Mixed,Mixed,"Infinity Ward, Raven Software, Beenox, Treyarc...","['FPS', 'Multiplayer', 'Shooter', 'Action', 'S...","['Single-player', 'Online PvP', 'Online Co-op'..."


In [6]:
cleaned_tags = steam_game_data
# removes the brackets and apostrophe from the list of tags
cleaned_tags["Popular Tags"] = cleaned_tags.loc[:, "Popular Tags"].str.replace("'","", regex=True)
cleaned_tags["Popular Tags"] = cleaned_tags.loc[:, "Popular Tags"].str.replace("\"","", regex=True)
cleaned_tags["Popular Tags"] = cleaned_tags.loc[:, "Popular Tags"].str.replace(r'\[|\]',"", regex=True)
cleaned_tags["Game Features"] = cleaned_tags.loc[:, "Game Features"].str.replace("'","", regex=True)
cleaned_tags["Game Features"] = cleaned_tags.loc[:, "Game Features"].str.replace("\"","", regex=True)
cleaned_tags["Game Features"] = cleaned_tags.loc[:, "Game Features"].str.replace(r'\[|\]',"", regex=True)
steam_game_data["Popular Tags"] = cleaned_tags["Popular Tags"]
steam_game_data["Game Features"] = cleaned_tags["Game Features"]
steam_game_data.head(5)

,Title,Release Year,Original Price,Recent Reviews Summary,All Reviews Summary,Developer,Popular Tags,Game Features
0,Baldur's Gate 3,"3 Aug, 2023",$29.99,Overwhelmingly Positive,Very Positive,Larian Studios,"RPG, Choices Matter, Character Customization, ...","Single-player, Online Co-op, LAN Co-op, Steam ..."
1,Counter-Strike: Global Offensive,"21 Aug, 2012",$14.99,Very Positive,Very Positive,"Valve, Hidden Path Entertainment","FPS, Shooter, Multiplayer, Competitive, Action...","Steam Achievements, Full controller support, S..."
2,Apex Legends™,"4 Nov, 2020",Free,Mixed,Very Positive,Respawn Entertainment,"Free to Play, Multiplayer, Battle Royale, Shoo...","Online PvP, Online Co-op, Steam Achievements, ..."
3,Forza Horizon 5,"8 Nov, 2021",$34.78,Very Positive,Very Positive,Playground Games,"Racing, Open World, Driving, Multiplayer, Auto...","Single-player, Online PvP, Online Co-op, Cross..."
4,Call of Duty®,"27 Oct, 2022",Free,Mixed,Mixed,"Infinity Ward, Raven Software, Beenox, Treyarc...","FPS, Multiplayer, Shooter, Action, Singleplaye...","Single-player, Online PvP, Online Co-op, Cross..."


In [7]:
cleaned_dates = steam_game_data
# still as a string
cleaned_dates["Release Year"] = cleaned_dates["Release Year"].str.replace(r".*,\s","", regex=True)
steam_game_data["Release Year"] = cleaned_dates["Release Year"]
steam_game_data.head(5)

,Title,Release Year,Original Price,Recent Reviews Summary,All Reviews Summary,Developer,Popular Tags,Game Features
0,Baldur's Gate 3,2023,$29.99,Overwhelmingly Positive,Very Positive,Larian Studios,"RPG, Choices Matter, Character Customization, ...","Single-player, Online Co-op, LAN Co-op, Steam ..."
1,Counter-Strike: Global Offensive,2012,$14.99,Very Positive,Very Positive,"Valve, Hidden Path Entertainment","FPS, Shooter, Multiplayer, Competitive, Action...","Steam Achievements, Full controller support, S..."
2,Apex Legends™,2020,Free,Mixed,Very Positive,Respawn Entertainment,"Free to Play, Multiplayer, Battle Royale, Shoo...","Online PvP, Online Co-op, Steam Achievements, ..."
3,Forza Horizon 5,2021,$34.78,Very Positive,Very Positive,Playground Games,"Racing, Open World, Driving, Multiplayer, Auto...","Single-player, Online PvP, Online Co-op, Cross..."
4,Call of Duty®,2022,Free,Mixed,Mixed,"Infinity Ward, Raven Software, Beenox, Treyarc...","FPS, Multiplayer, Shooter, Action, Singleplaye...","Single-player, Online PvP, Online Co-op, Cross..."


In [8]:
cleaned_price = steam_game_data
# still as a string
cleaned_price["Original Price"] = cleaned_price["Original Price"].str.replace(r"Free", "0", regex=True)
cleaned_price["Original Price"] = cleaned_price["Original Price"].str.replace(r"\$", "", regex=True)
steam_game_data["Original Price"] = cleaned_price["Original Price"]
steam_game_data

,Title,Release Year,Original Price,Recent Reviews Summary,All Reviews Summary,Developer,Popular Tags,Game Features
0,Baldur's Gate 3,2023,29.99,Overwhelmingly Positive,Very Positive,Larian Studios,"RPG, Choices Matter, Character Customization, ...","Single-player, Online Co-op, LAN Co-op, Steam ..."
1,Counter-Strike: Global Offensive,2012,14.99,Very Positive,Very Positive,"Valve, Hidden Path Entertainment","FPS, Shooter, Multiplayer, Competitive, Action...","Steam Achievements, Full controller support, S..."
2,Apex Legends™,2020,0,Mixed,Very Positive,Respawn Entertainment,"Free to Play, Multiplayer, Battle Royale, Shoo...","Online PvP, Online Co-op, Steam Achievements, ..."
3,Forza Horizon 5,2021,34.78,Very Positive,Very Positive,Playground Games,"Racing, Open World, Driving, Multiplayer, Auto...","Single-player, Online PvP, Online Co-op, Cross..."
4,Call of Duty®,2022,0,Mixed,Mixed,"Infinity Ward, Raven Software, Beenox, Treyarc...","FPS, Multiplayer, Shooter, Action, Singleplaye...","Single-player, Online PvP, Online Co-op, Cross..."
...,...,...,...,...,...,...,...,...
37019,O2Jam Online,2023,0,Very Negative,Overwhelmingly Negative,VALOFE,"Casual, Rhythm, Relaxing, Colorful, Score Atta...","Single-player, In-App Purchases"
37971,Call of Duty®: Warzone™,2022,0,Mostly Negative,Mostly Negative,"Infinity Ward, Raven Software, Beenox, Treyarc...","Free to Play, Battle Royale, Multiplayer, FPS,...","Single-player, Online PvP, Online Co-op, Cross..."
37973,Their Land,2023,0,Mixed,Mostly Negative,Thomas,"Singleplayer, Adventure, Action, Action-Advent...","Single-player, Steam Achievements, Partial Con..."
49113,Call of Duty®: Modern Warfare® II,2022,69.99,Overwhelmingly Negative,Overwhelmingly Negative,"Infinity Ward, Raven Software, Beenox, Treyarc...","Action, FPS, Multiplayer, Singleplayer, Shoote...","Single-player, Online PvP, Online Co-op, Cross..."


In [9]:
encoded_tags = steam_game_data

# encodes the Popular Tags, Developer, and Game Features with one hot encoding
encoded_tags_popular = encoded_tags['Popular Tags'].str.get_dummies(sep=', ')
encoded_tags_features = encoded_tags['Game Features'].str.get_dummies(sep=', ')
encoded_tags_develoepr = encoded_tags['Developer'].str.get_dummies(sep=', ')

game_data = pd.concat([encoded_tags.drop('Popular Tags', axis=1), encoded_tags_popular], axis=1)
game_data = pd.concat([game_data.drop('Game Features', axis=1), encoded_tags_features], axis=1)
game_data = pd.concat([game_data.drop('Developer', axis=1), encoded_tags_develoepr], axis=1)

game_data


,Title,Release Year,Original Price,Recent Reviews Summary,All Reviews Summary,1980s,1990s,2.5D,2D,2D Fighter,...,谌嘉诚,赤瞳大白猫,锁眼工作室,阿正不经,香港商河洛互動娛樂股份有限公司,高考恋爱委员会,鬼谷工作室,공감오래 콘텐츠,신인건,（Hong Kong）GKD
0,Baldur's Gate 3,2023,29.99,Overwhelmingly Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Counter-Strike: Global Offensive,2012,14.99,Very Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Apex Legends™,2020,0,Mixed,Very Positive,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Forza Horizon 5,2021,34.78,Very Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Call of Duty®,2022,0,Mixed,Mixed,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37019,O2Jam Online,2023,0,Very Negative,Overwhelmingly Negative,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37971,Call of Duty®: Warzone™,2022,0,Mostly Negative,Mostly Negative,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37973,Their Land,2023,0,Mixed,Mostly Negative,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49113,Call of Duty®: Modern Warfare® II,2022,69.99,Overwhelmingly Negative,Overwhelmingly Negative,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
game_data.to_csv("processed_games.csv", index=False)

In [ ]:
review_file = games_path + "/merged_data.csv"

uncleaned_review_data = pd.read_csv(review_file)
uncleaned_review_data.head(5)

In [ ]:
filter_review_data = uncleaned_review_data

# votes_up is the users binary review of the game [recommend, does not recommend]
filter_review_data = filter_review_data[filter_review_data["votes_up"] >= 10]

# filters so users must have at least 30 minuites in the game
filter_review_data = filter_review_data[filter_review_data["author_playtime_at_review"] >= 30]

# filters entries so that only reviews are kept if the user has more than 1 review within the dataset
# this doesn't use author_num_reviews since we are currently starting with a sub dataset from out of the 110 million entrie dataset
filter_review_data = filter_review_data.loc[filter_review_data.groupby('author_steamid')['author_steamid'].transform('size') > 1]

# matches the games that exist in both datasets and drops the ones that are not 
filter_review_data = filter_review_data[filter_review_data["game"].isin(game_data["Title"])]
filter_review_data = filter_review_data.rename(columns={"game": "Title"})

review_data = filter_review_data[["appid", "Title", "author_steamid", "author_playtime_at_review", "voted_up", "votes_up", "weighted_vote_score"]]

,recommendationid,appid,Title,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
613,140867490,1000010,Crown Trick,76561198420455365,0,13,2219,0,2219,1667502748,...,1,117,14,0.914299,1,0,0,0,1,NaN
620,79733845,1000010,Crown Trick,76561198928089084,60,9,127,0,44,1614865472,...,1,156,410,0.810662,1,1,0,0,0,NaN
623,77899125,1000010,Crown Trick,76561198263458033,0,10,1753,0,1753,1603241500,...,1,117,7,0.837473,8,1,0,0,0,NaN
624,77897266,1000010,Crown Trick,76561198064031065,0,79,2088,0,93,1621674578,...,0,64,0,0.811944,0,1,0,0,0,NaN
625,77761663,1000010,Crown Trick,76561198139548920,0,92,3101,0,268,1635556935,...,0,253,23,0.847576,13,1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498067,94282514,997010,Police Simulator: Patrol Officers,76561198800636906,477,49,931,0,359,1636195998,...,1,1149,506,0.982088,1,0,0,0,0,NaN
498068,94192417,997010,Police Simulator: Patrol Officers,76561198125878506,0,31,552,0,322,1624644069,...,1,667,933,0.975688,0,1,0,1,0,NaN
498071,94134146,997010,Police Simulator: Patrol Officers,76561198797537147,97,47,2972,0,2823,1673658359,...,1,329,14,0.952807,7,1,0,0,0,NaN
498077,94008459,997010,Police Simulator: Patrol Officers,76561198020487283,5792,965,243,0,243,1624095561,...,1,99,7,0.897610,0,0,0,1,0,NaN


In [ ]:
review_data.to_csv("processed_user_review_data.csv", index=False)

In [ ]:
merged_data = pd.merge(game_data, review_data, on="Title")
merged_data.head(5)

,Title,Release Year,Original Price,Recent Reviews Summary,All Reviews Summary,1980s,1990s,2.5D,2D,2D Fighter,...,鬼谷工作室,공감오래 콘텐츠,신인건,（Hong Kong）GKD,appid,author_steamid,author_playtime_at_review,voted_up,votes_up,weighted_vote_score
0,Baldur's Gate 3,2023,29.99,Overwhelmingly Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,1086940,76561198048832862,14294,1,68,0.833663
1,Baldur's Gate 3,2023,29.99,Overwhelmingly Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,1086940,76561198268696214,10296,1,79,0.860387
2,Baldur's Gate 3,2023,29.99,Overwhelmingly Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,1086940,76561198214987871,9782,1,57,0.828677
3,Baldur's Gate 3,2023,29.99,Overwhelmingly Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,1086940,76561198032500682,19421,1,137,0.837772
4,Baldur's Gate 3,2023,29.99,Overwhelmingly Positive,Very Positive,0,0,0,0,0,...,0,0,0,0,1086940,76561198065068255,7264,1,45,0.807603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81249,HAWKEN REBORN,2023,0,Mostly Negative,Mostly Negative,0,0,0,0,0,...,0,0,0,0,705040,76561197970359498,166,0,79,0.832614
81250,HAWKEN REBORN,2023,0,Mostly Negative,Mostly Negative,0,0,0,0,0,...,0,0,0,0,705040,76561197990917469,74,0,564,0.932847
81251,Call of Duty®: Warzone™,2022,0,Mostly Negative,Mostly Negative,0,0,0,0,0,...,0,0,0,0,1962663,76561198048832362,35,1,539,0.891991
81252,Call of Duty®: Warzone™,2022,0,Mostly Negative,Mostly Negative,0,0,0,0,0,...,0,0,0,0,1962663,76561198338776245,251,1,1633,0.895560


In [ ]:
# Creates nodes for each games in the dataset
game_ids = merged_data[["appid", "Title"]].drop_duplicates()

game_nodes = pd.merge(
    game_data,
    game_ids,
    on="Title",
    how="left"
)

game_nodes = game_nodes.rename(columns={"appid": "game_id"})
game_nodes = game_nodes.drop(columns=["Title"])
game_nodes.to_csv("game_nodes.csv", index=False)

In [ ]:
user_nodes = merged_data[["author_steamid"]].drop_duplicates()
user_nodes = user_nodes.rename(columns={"author_steamid": "user_id"})

user_nodes.to_csv("user_nodes.csv", index=False)
print("ser_nodes.csv saved:", user_nodes.shape)

ser_nodes.csv saved: (33918, 1)


In [ ]:
edges = merged_data[["author_steamid", "appid", "voted_up"]].copy()

edges = edges.rename(columns={
    "author_steamid": "user_id",
    "appid": "game_id",
    "voted_up": "rating"
})

edges.to_csv("edges.csv", index=False)
print("edges.csv saved:", edges.shape)

edges.csv saved: (81254, 3)


In [ ]:
print("Users missing from user_nodes:", (~edges["user_id"].isin(user_nodes["user_id"])).sum())
print("Games missing from game_nodes:", (~edges["game_id"].isin(game_nodes["game_id"])).sum())

Users missing from user_nodes: 0
Games missing from game_nodes: 0


In [ ]:
import torch
from torch_geometric.data import HeteroData

def convert_dataframe_to_heterodata(df: pd.DataFrame):
    # 1. Map string IDs to 0-based integer indices
    
    unique_users = df['user_id'].unique()
    user_mapping = {id: i for i, id in enumerate(unique_users)}
    num_users = len(unique_users)

    unique_games = df['game_id'].unique()
    game_mapping = {id: i for i, id in enumerate(unique_games)}
    num_games = len(unique_games)

    # 2. Prepare Tensors for Edge Index and Attributes
    
    # Source nodes (Users): indices 0 to N_users-1
    user_indices = torch.tensor(
        df['user_id'].map(user_mapping).values, 
        dtype=torch.long
    )

    # Target nodes (Games): indices 0 to N_games-1
    game_indices = torch.tensor(
        df['game_id'].map(game_mapping).values, 
        dtype=torch.long
    )
    
    # Edge Label (Rating)
    ratings = torch.tensor(df['rating'].values, dtype=torch.float)

    # 3. Create HeteroData Object
    data = HeteroData()

    # Define node types and their number
    data['user'].num_nodes = num_users
    data['game'].num_nodes = num_games
    
    # Define the edge type: ('user', 'rates', 'game')
    data['user', 'rates', 'game'].edge_index = torch.stack(
        [user_indices, game_indices], 
        dim=0
    )

    # Add the rating as the edge label (y)
    data['user', 'rates', 'game'].y = ratings
    
    return data


In [ ]:
pyg_data = convert_dataframe_to_heterodata(edges)
print(pyg_data)

HeteroData(
  user={ num_nodes=33918 },
  game={ num_nodes=3761 },
  (user, rates, game)={
    edge_index=[2, 81254],
    y=[81254],
  }
)


In [ ]:
import torch
from torch_geometric.transforms import RandomLinkSplit

# Define the link type to be split ('user', 'rates', 'game')
edge_type = ('user', 'rates', 'game')

# Instantiate the link splitter
# is_undirected=False is essential for user-item recommendation graphs
# add_negative_train_samples=False is typically used here, as we sample negative edges 
# later, usually inside the training loop or a custom sampler.
transform = RandomLinkSplit(
    num_val=0.1,  # 10% of edges for validation
    num_test=0.1, # 10% of edges for testing
    is_undirected=False,
    neg_sampling_ratio=0.0, # Do not sample negatives yet (easier to handle later)
    edge_types=[edge_type],
    rev_edge_types=[edge_type[::-1]] # Define the reverse edge type for bipartite graphs
)

# Apply the transform to create the train/val/test splits
# NOTE: The output are three separate HeteroData objects
train_data, val_data, test_data = transform(pyg_data)

# Print to confirm the splits
print(f"Original Edges: {pyg_data[edge_type].edge_index.size(1)}")
# Edges remaining in the training graph structure (should be ~80%)
print(f"Train Edges (Graph Structure): {train_data[edge_type].edge_index.size(1)}")

# Positive links extracted for Validation (should be ~10%)
print(f"Validation Target Links: {val_data[edge_type].edge_label_index.size(1)}")

# Positive links extracted for Testing (should be ~10%)
print(f"Test Target Links: {test_data[edge_type].edge_label_index.size(1)}")

Original Edges: 81254
Train Edges (Graph Structure): 65004
Validation Target Links: 8125
Test Target Links: 8125
